In [74]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer

from html.parser import HTMLParser

In [2]:
## data with the real text meat
prof = pd.read_csv("../professionals.csv")
answ = pd.read_csv("../answers.csv")
stud = pd.read_csv("../students.csv")
ques = pd.read_csv("../questions.csv")

In [11]:
# Professional's headlines - clean them!!!
def clean_names(headline): 
    if type(headline) == str:
        return re.sub(r'at\s.*$', '', headline)
    return headline

# Updated the city columns 
prof.professionals_headline = prof.professionals_headline.apply(clean_names)
prof.sample(2)

,professionals_id,professionals_location,professionals_industry,professionals_headline,professionals_date_joined
28118,de2415064b9b445c8717425ed70fd99a,"El Segundo, California",Entertainment,Software Management and Technical Project Mana...,2019-01-31 20:06:51 UTC+0000
27060,435f53c586fd42e4ae95fb202bfa99bd,"Boston, Massachusetts",www.linkedin.com/in/jyotsna-penumaka,Computer Engineering,2019-01-09 14:42:28 UTC+0000


In [16]:
## Function to get rid of HTML tags from text
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(str(d))
    def get_data(self):
        return ''.join(self.fed)
    
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [45]:
# Clean up these bodies of text

# Tag parts of speech and remove stop words
# More interested in knowing which general area these questions are asking about so let's grab nouns
stop_words = set(stopwords.words("english"))
def filterText(text):
    # initialize stop words bank
    stop_words = set(stopwords.words("english"))
    
    # strip html! 
    text = strip_tags(str(text).lower())
    text = (nltk.word_tokenize(text))
    #print('Finished tokenizing body text')
    
    text = nltk.pos_tag(text)
    #print('Finished taging parts of speech')
    
    res = ""
    for i in text: 
        if (i[0] not in stop_words) and (i[1] == "NN" or i[1] == "NNP" or i[1] == "NNS" or i[1] == "NNPS"):
            res += i[0] + " "
    return res
print(ques.sample(3))
ques.questions_body = ques.questions_body.apply(filterText)


                           questions_id               questions_author_id  \
6522   cb950ef4249745f8a3fec031db3442a5  d511e5160d464de581b909ee7362ccfc   
13011  358a91e9be23441f8fc3eadae25c5c68  b689079a498740bdb416f29894165cbc   
1273   9175ac1c42dc4950a6437591a53b3342  6c04ae6e95044e1e9cd2521d82b9e030   

               questions_date_added  \
6522   2015-06-26 22:44:43 UTC+0000   
13011  2016-05-05 06:06:17 UTC+0000   
1273   2016-05-12 18:08:47 UTC+0000   

                                         questions_title  \
6522   How do i become a event planner for party's at...   
13011  I want to become a teacher , which are the sub...   
1273           What is it like in an entrepreneur class?   

                                          questions_body  
6522          years event planner skill lot money world   
13011                hobes sports dance and9th educator   
1273   business therapy clinic athletes california bu...  


In [49]:
print(ques.sample(2))
## Sweet! It works. Now we can apply some other tools to this cleaned text

                           questions_id               questions_author_id  \
20060  95061841221e465d95b35a7b36e1f0dd  631f93de8e8f429385dab2542b1e13a3   
15760  eb353da4974f4eae9f2fa7a766cf8108  eeeb3ea607f14bc28da6a5379a01796d   

               questions_date_added  \
20060  2016-05-18 03:32:08 UTC+0000   
15760  2015-10-26 15:56:07 UTC+0000   

                                         questions_title  \
20060  Do you have to go to an artsy school to end up...   
15760                  how hard is it to be an engineer?   

                                          questions_body  
20060  jesuit school education design program company...  
15760              engineer engineering engineer skills   


In [98]:
## Grab a corpus of text as a numpy array instead of series
text = ques.questions_body.values
type(text)

numpy.ndarray

In [96]:
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv.fit_transform(text)
text_counts = text_counts.toarray()
vocab = cv.get_feature_names()
text_counts = pd.DataFrame(text_counts, columns=vocab)
text_counts.sample(3)

,1,12,12education,12th,14,16,22,3,39,3d,...,zines,zip,zombie,zone,zoo,zookeeper,zookeepers,zoologist,zoology,zymeria
18487,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(text)
tv_matrix = tv_matrix.toarray()
vocab = tv.get_feature_names()
tfDF = pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)
tfDF.sample(4)

,12,12education,12th,14,16,22,39,3d,3h9xfknaqr4,891,...,zines,zip,zombie,zone,zoo,zookeeper,zookeepers,zoologist,zoology,zymeria
4698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
